In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import time
import datetime
import pickle
import gc

In [2]:
n_fold = 10

In [3]:
models_dir = 'models/'

In [4]:
names = ['Silver', 'Gold', 'Copper', 'Palladium', 'Platinum']

In [5]:
lags = [10, 20, 50]

In [6]:
models = [[[] for split in range(n_fold)] for name in names]

for n in range(len(names)):
    for split in range(n_fold):
        models[n][split] = pickle.load(open(f'{models_dir}/trained_fullmetal_model_id_{names[n]}_fold{str(split)}.pkl', 'rb'))

In [7]:
len(models),len(models[0])

(5, 10)

In [8]:
gc.collect()

0

In [9]:
TRAIN_CSV = 'train_metals_features.csv'

In [10]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [11]:
df_train = pd.read_csv(TRAIN_CSV)

In [12]:
df_train.drop(columns='Unnamed: 0', inplace=True)

In [13]:
#df_train = reduce_mem_usage(df_train)
train_merged = df_train.copy()

not_use_features_train = ['Date']

for name in names:
    not_use_features_train.append(f'Price_{name}')

features = train_merged.drop(columns=not_use_features_train).columns.to_list()
print(len(features), features)

108 ['Price_Lag_10_Silver', 'log_close/mean_10_Silver', 'log_return_10_Silver', 'Price_Lag_20_Silver', 'log_close/mean_20_Silver', 'log_return_20_Silver', 'Price_Lag_50_Silver', 'log_close/mean_50_Silver', 'log_return_50_Silver', 'mean_close/mean_10_Silver', 'mean_log_returns_10_Silver', 'log_close/mean_10-mean_close/mean_10_Silver', 'log_return_10-mean_log_returns_10_Silver', 'mean_close/mean_20_Silver', 'mean_log_returns_20_Silver', 'log_close/mean_20-mean_close/mean_20_Silver', 'log_return_20-mean_log_returns_20_Silver', 'mean_close/mean_50_Silver', 'mean_log_returns_50_Silver', 'log_close/mean_50-mean_close/mean_50_Silver', 'log_return_50-mean_log_returns_50_Silver', 'Year', 'Month', 'Week_Number', 'Price_Lag_10_Gold', 'log_close/mean_10_Gold', 'log_return_10_Gold', 'Price_Lag_20_Gold', 'log_close/mean_20_Gold', 'log_return_20_Gold', 'Price_Lag_50_Gold', 'log_close/mean_50_Gold', 'log_return_50_Gold', 'mean_close/mean_10_Gold', 'mean_log_returns_10_Gold', 'log_close/mean_10-mean_cl

In [14]:
train_merged.dtypes.unique()

array([dtype('float64'), dtype('O'), dtype('int64')], dtype=object)

In [15]:
train_merged = reduce_mem_usage(train_merged)

Memory usage of dataframe is 2.16 MB
Memory usage after optimization is: 0.62 MB
Decreased by 71.4%


In [16]:
train_merged['Date']

0       2013-04-09
1       2013-04-10
2       2013-04-11
3       2013-04-12
4       2013-04-15
           ...    
2474    2023-02-22
2475    2023-02-23
2476    2023-02-24
2477    2023-02-27
2478    2023-02-28
Name: Date, Length: 2479, dtype: category
Categories (2479, object): ['2013-04-09', '2013-04-10', '2013-04-11', '2013-04-12', ..., '2023-02-23', '2023-02-24', '2023-02-27', '2023-02-28']

In [17]:
def get_lags(df):
    lags = [10,20,50]
    df_out = pd.DataFrame()

    for col in df.columns:

        for lag in lags:
            df_out[f'{col}_Lag_{lag}'] = df[col].shift(lag)
            df_out[f'log_close/mean_{lag}'] = np.log( np.array(df[col]) /  np.roll(np.append(np.convolve( np.array(df[col]), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1))
            df_out[f'log_return_{lag}']     = np.log( np.array(df[col]) /  np.roll(np.array(df[col]), lag))

        for lag in lags:
            df_out[f'mean_close/mean_{lag}'] =  np.mean(df_out.iloc[:,df_out.columns.str.startswith(f'log_close/mean_{lag}')], axis=1)
            df_out[f'mean_log_returns_{lag}'] = np.mean(df_out.iloc[:,df_out.columns.str.startswith(f'log_return_{lag}')] ,    axis=1)

            df_out[f'log_close/mean_{lag}-mean_close/mean_{lag}'] = np.array( df_out[f'log_close/mean_{lag}']) - np.array( df_out[f'mean_close/mean_{lag}']  )
            df_out[f'log_return_{lag}-mean_log_returns_{lag}']    = np.array( df_out[f'log_return_{lag}'])     - np.array( df_out[f'mean_log_returns_{lag}'] )

    return df_out

def get_date_features(df):
    df_copy = df.copy()
    df_copy['Date'] = pd.to_datetime(df_copy['Date'])
    df_copy['Year'] = df_copy['Date'].dt.year
    df_copy['Month'] = df_copy['Date'].dt.month
    df_copy['Week_Number'] = df_copy['Date'].dt.isocalendar().week
    return df_copy

def get_features(df, max_date):
    #df_feat = df[['Price', 'Change_%', 'MA_for_10_days', 'MA_for_20_days', 'MA_for_50_days']].copy() # 'Open', 'High', 'Low', , 
    #df_feat = pd.concat([df_feat, get_lags(df_feat[['MA_for_10_days', 'MA_for_20_days', 'MA_for_50_days']])], axis=1)
    #df_feat['Upper Shadow'] = upper_shadow(df_feat)
    #df_feat['Lower Shadow'] = lower_shadow(df_feat)
    
    # Time window
    df_range = df[df['Date'] >= max_date]

    # Fatures to lag
    df_feat = get_lags(df_range[['Price']])

    # Date features
    date_df = df_range[['Date']].copy()
    date_df['Date'] = pd.to_datetime(date_df['Date'])
    df_feat = pd.concat([df_feat, get_date_features(date_df), df_range[['Price']]], axis=1)
    df_feat.dropna(inplace=True)
    return df_feat

In [18]:
train_merged['Date'] = pd.to_datetime(train_merged['Date'])

In [19]:
train_merged.iloc[-1]['Date'], train_merged.iloc[-1]['Date'] + pd.DateOffset(days=1)

(Timestamp('2023-02-28 00:00:00'), Timestamp('2023-03-01 00:00:00'))

In [20]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

date_range = pd.date_range(
    train_merged.iloc[-1]['Date'] + pd.DateOffset(days=1), 
    train_merged.iloc[-1]['Date'] + pd.DateOffset(days=30), freq=us_bd)
date_df = pd.DataFrame({'Date': date_range})
date_df

,Date
0,2023-03-01
1,2023-03-02
2,2023-03-03
3,2023-03-06
4,2023-03-07
5,2023-03-08
6,2023-03-09
7,2023-03-10
8,2023-03-13
9,2023-03-14


- Strategy: 
    the model is trained with lags of 10, 20, and 50 days.
    it's also trained using cv of 10, which means, it uses about 9 years of training, and 1 year of data for testing, because the date range is from 2013 to 2023.
    we have 2 types of models, the ones trained with 5 metals standalone, and for each one we have 10 models, totalling 50 models.
    and also 50 more models trained with the whole dataset, so it can see how the other metals behave, which gives it a more general view over the market.

    1) Load the data and generate a future window of time (in us business days).
    2) 